In [176]:
import numpy as np
from skimage import io
import math
from matplotlib import pyplot as plt

In [78]:
happy = ['0105', '0107', '0110']
neutral = ['0101', '0106', '0109']
sad = ['0106', '0107', '0110']

In [79]:
def create_pixel_matrix(emotion: str, number_images: list) -> list:
    images_pixel = []
    for i in range(len(number_images)):
        images_pixel.append(io.imread(f'datasets/{emotion}/{emotion}-{number_images[i]}.jpg', as_gray = True))
    return images_pixel

In [80]:
def show_histogram(emotion: str, number_images: list, image_pixel: list):
    for i in range(len(image_pixel)):
        plt.subplot(1, 2, 1)
        plt.imshow(image_pixel[i], cmap = 'gray')
        plt.title(f'{emotion}-{number_images[i]}')
        
        plt.subplot(1, 2, 2)
        plt.hist(image_pixel[i].ravel(), 256, [0, 255])
        plt.title(f'{emotion}-{number_images[i]} histogram')

        plt.show()

In [213]:
def first_order_statistics(image_pixel: list):
    hist, bins = np.histogram(image_pixel, 256, [0, 255])

    # mean
    # print(np.mean(image_pixel))
    hist_sum = sum(hist)
    hist_per_sum = []
    mean = 0
    for h in hist:
        hist_per_sum.append(h / hist_sum)
    for i in range(256):
        mean += hist_per_sum[i] * i

    print(mean)

    # variance
    # print(np.var(image_pixel))
    variance = 0
    for i in range(256):
        variance += (i - mean) ** 2 * hist_per_sum[i]
    print(variance)

    # skewness
    skewness = 0
    for i in range(256):
        skewness += (i - mean) ** 3 * hist_per_sum[i]
    skewness = skewness / (variance * math.sqrt(variance))
    print(skewness)

    # kurtosis
    kurtosis = 0
    for i in range(256):
        kurtosis += (i - mean) ** 4 * hist_per_sum[i]
    kurtosis = kurtosis / (variance ** 2) - 3
    print(kurtosis)

    # entropy
    entropy = 0
    for i in range(246):
        entropy += hist_per_sum[i] * math.log2(np.spacing(hist_per_sum[i]))
    entropy = -entropy
    print(entropy)

In [214]:
pixel_image = create_pixel_matrix('happy', happy)
# show_histogram('happy', happy, pixel_image)
first_order_statistics(pixel_image[0])

102.97612847222233
1664.5502114001613
0.7117525093143018
0.23420158205262442
59.60503472222219
